# Goni_Wind_10m

### This notebook uses [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/) to display an animated wind velocity flow field, a la https://earth.nullschool.net or  https://www.windy.com/ 

#### Original notebook made November 2020 with python == 3.7.6, xarray == 0.16.0 and ipyleaflet == 0.13.1

In [1]:
import datetime
from ipyleaflet import Map, TileLayer, basemaps
from ipyleaflet.velocity import Velocity
import xarray as xr

### Read in a dataset containing 0-hour forecast data from the 0.25 degree GFS around the time of Goni's landfall.

In [2]:
ds = xr.open_dataset('http://thredds.atmos.albany.edu:8080/thredds/dodsC/GRIB2/goni/goni_gfs.grib2')

In [3]:
# Examine the dataset
ds

<xarray.Dataset>
Dimensions:                                                          (altitude_above_msl: 3, height_above_ground: 2, height_above_ground1: 6, hybrid: 1, isobaric: 31, isobaric1: 1, isobaric2: 22, isobaric3: 21, isobaric4: 26, isobaric5: 17, isobaric6: 25, lat: 721, lon: 1440, potential_vorticity_surface: 2, pressure_difference_layer: 1, pressure_difference_layer1: 1, pressure_difference_layer1_bounds_1: 2, pressure_difference_layer_bounds_1: 2, sigma: 1, sigma_layer: 4, sigma_layer_bounds_1: 2, time: 9)
Coordinates:
  * lat                                                              (lat) float32 ...
  * lon                                                              (lon) float32 ...
  * time                                                             (time) datetime64[ns] ...
    reftime                                                          (time) datetime64[ns] ...
  * isobaric                                                         (isobaric) float32 ...
  * sigma                                                            (sigma) float32 ...
  * pressure_difference_layer                                        (pressure_difference_layer) float32 ...
  * isobaric1                                                        (isobaric1) float32 ...
  * isobaric2                                                        (isobaric2) float32 ...
  * isobaric3                                                        (isobaric3) float32 ...
  * pressure_difference_layer1                                       (pressure_difference_layer1) float32 ...
  * altitude_above_msl                                               (altitude_above_msl) float32 ...
  * height_above_ground                                              (height_above_ground) float32 ...
  * height_above_ground1                                             (height_above_ground1) float32 ...
  * isobaric4                                                        (isobaric4) float32 ...
  * hybrid                                                           (hybrid) float32 ...
  * sigma_layer                                                      (sigma_layer) float32 ...
  * isobaric5                                                        (isobaric5) float32 ...
  * potential_vorticity_surface                                      (potential_vorticity_surface) float32 ...
  * isobaric6                                                        (isobaric6) float32 ...
Dimensions without coordinates: pressure_difference_layer1_bounds_1, pressure_difference_layer_bounds_1, sigma_layer_bounds_1
Data variables:
    LatLon_Projection                                                int32 ...
    pressure_difference_layer_bounds                                 (pressure_difference_layer, pressure_difference_layer_bounds_1) float32 ...
    pressure_difference_layer1_bounds                                (pressure_difference_layer1, pressure_difference_layer1_bounds_1) float32 ...
    sigma_layer_bounds                                               (sigma_layer, sigma_layer_bounds_1) float32 ...
    Absolute_vorticity_isobaric                                      (time, isobaric4, lat, lon) float32 ...
    Cloud_mixing_ratio_isobaric                                      (time, isobaric2, lat, lon) float32 ...
    Cloud_mixing_ratio_hybrid                                        (time, hybrid, lat, lon) float32 ...
    Cloud_water_entire_atmosphere_single_layer                       (time, lat, lon) float32 ...
    Convective_available_potential_energy_pressure_difference_layer  (time, pressure_difference_layer1, lat, lon) float32 ...
    Convective_available_potential_energy_surface                    (time, lat, lon) float32 ...
    Convective_inhibition_surface                                    (time, lat, lon) float32 ...
    Convective_inhibition_pressure_difference_layer                  (time, pressure_difference_layer1, lat, lon) float32 ...
    Geopotential_height_isobaric  

In [4]:
# Select the 31 October 2020 1800 UTC analysis time and the 20m AGL level (this is the lowest level in the dataset)
u = ds['u-component_of_wind_height_above_ground'].sel(time='2020-10-31-18:00',height_above_ground1=20)
v = ds['v-component_of_wind_height_above_ground'].sel(time='2020-10-31-18:00',height_above_ground1=20)

### Create a Dataset that contains just the 20m U and V wind components, since that's what we'll pass to the `Velocity` method of ipyleaflet.
(Perhaps there is a more efficient way of doing this?)

In [5]:
# First create the Dataset out of the 'u' Dataarray
uv = u.to_dataset()

In [6]:
# Append v to the Dataset
uv['v'] = v

In [7]:
# Rename the u DataArray to 'u'
name_dict = {'u-component_of_wind_height_above_ground':'u'}
uv = uv.rename_vars(name_dict)

In [8]:
# Verify that the Dataset looks ok
uv

<xarray.Dataset>
Dimensions:               (lat: 721, lon: 1440)
Coordinates:
  * lat                   (lat) float32 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * lon                   (lon) float32 0.0 0.25 0.5 ... 359.25 359.5 359.75
    time                  datetime64[ns] 2020-10-31T18:00:00
    reftime               datetime64[ns] 2020-10-31T18:00:00
    height_above_ground1  float32 20.0
Data variables:
    u                     (lat, lon) float32 ...
    v                     (lat, lon) float32 ...

### Create an ipyleaflet map object; center it, specify a zoom level, and select a background map; create and add a velocity layer from the Dataset that contains u and v.

In [9]:
center = (13.25, 124.75)
zoom = 6
m = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.CartoDB.DarkMatter)


display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(data=uv,
                zonal_speed='u',
                meridional_speed='v',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=120,
                display_options=display_options)
m.add_layer(wind)

### Display the map.

In [10]:
m

Map(center=[13.25, 124.75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…